In [56]:
%load_ext autoreload
%autoreload 2

import pandas as pd  
import numpy as np
import math
from datetime import datetime
from IPython.display import display, HTML

from funcoes import *
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

threshold = 30
alpha = 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
#dict with coordinates
coord_gauges = '/home/mary/mc/IDW method/consisted_stations_XY_ok.xlsx'
df_coord = pd.read_excel(coord_gauges)
#df_coord = df_coord.drop(['Name'], axis=1)
# transform gauge names from int to strings
df_coord['Gauge'] = df_coord['Gauge'].astype(str)
coords = df_coord.set_index('Gauge').T.to_dict()

In [58]:
#rainfall timeseries
data = '/home/mary/mc/IDW method/consisted_rain_gauges.xlsx'
gauge_data = pd.read_excel(data)
gauge_data = gauge_data.set_index(['Date'])
# transform gauge names from int to strings
gauge_data.columns = [str(c) for c in gauge_data.columns]

In [59]:
filter_gauges = gauge_data.notna().dot(gauge_data.columns+',').str.rstrip(',')
filter_gauges = pd.DataFrame(filter_gauges)
filter_gauges.index = pd.to_datetime(filter_gauges.index)
df = pd.DataFrame(filter_gauges).reset_index()
groups = df.groupby(0)['Date'].apply(list).reset_index(name='dias')
groups.columns = ['gauges', 'dias']
#groups['dias']

In [60]:
meus_dias = ['2000-06-07', '2000-06-08']
gauge_data.loc[gauge_data.index.isin(meus_dias)]

,1954003,1456005,1853002,2157005,1455014,1755003,1654005,1757003,1957003,2054009,1456001,1854003,1656002,1454003,1957005,1957006,1556000,2256001,1457004,1657004,1655001,1658000,1555005,2055001,1558004,2257000,1558005,1757001,1555010,1754000,1556007,2056003,1956008,1555012,1655002,1957004,1457000,1857000,2157003,1956003,1857003,1854002,2056001,1654000,2057001,1854001,1754002,2055003,1655003,1853005,2056007,2054021,1659001,2156001,1656003,2055004,1956001,2055002,1555007,1654004,2257001,1857001,1455008,1556010,1456003,1956004,1555001,1656004,1755000,1456008,1455011,2054019,1557003,1656001,1655004,1557000,1555008,1557001,2155001,1454002,1556006,2054005,1555011,1854004,1854006,2156000,2157006,1654001,1655000,1657003,1558000,1557005,1954006,1556005,1455012,2057000,1954004,1457001,1558001,1455010,1456004,2157004,1955000
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-06-07,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-06-08,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.1,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,3.2,0.0


In [61]:
dfs = []
for dias in groups['dias']:
    meus_dias = [dia.strftime("%Y-%m-%d") for dia in dias]
    df = gauge_data.loc[gauge_data.index.isin(meus_dias)]
    dfs.append(df)

In [62]:
# get the names of gauges w/ values at each line of gauge_data
filter_gauges = gauge_data.notna().dot(gauge_data.columns+',').str.rstrip(',')
dfs = pd.DataFrame(filter_gauges)

In [63]:
gauge_data

,1954003,1456005,1853002,2157005,1455014,1755003,1654005,1757003,1957003,2054009,1456001,1854003,1656002,1454003,1957005,1957006,1556000,2256001,1457004,1657004,1655001,1658000,1555005,2055001,1558004,2257000,1558005,1757001,1555010,1754000,1556007,2056003,1956008,1555012,1655002,1957004,1457000,1857000,2157003,1956003,1857003,1854002,2056001,1654000,2057001,1854001,1754002,2055003,1655003,1853005,2056007,2054021,1659001,2156001,1656003,2055004,1956001,2055002,1555007,1654004,2257001,1857001,1455008,1556010,1456003,1956004,1555001,1656004,1755000,1456008,1455011,2054019,1557003,1656001,1655004,1557000,1555008,1557001,2155001,1454002,1556006,2054005,1555011,1854004,1854006,2156000,2157006,1654001,1655000,1657003,1558000,1557005,1954006,1556005,1455012,2057000,1954004,1457001,1558001,1455010,1456004,2157004,1955000
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-06-02,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,5.6,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,1.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-06-03,0.0,0.0,0.0,0.3,NaN,0.0,0.0,0.0,1.3,4.1,0.0,0.0,0.0,NaN,0.4,16.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.8,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,2.4,5.7,0.0,1.2,0.0,1.1,0.0,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.5,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,NaN,0.0,0.0,5.5,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,72.0,3.0
2000-06-04,0.0,0.0,0.0,69.0,NaN,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,NaN,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8,0.0,38.0,0.0,0.0,NaN,0.0,0.0,17.4,NaN,NaN,0.0,2.5,0.0,4.1,18.9,0.0,0.6,0.0,8.3,0.0,17.0,0.0,0.0,5.8,0.0,0.0,20.2,0.0,0.0,33.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.6,0.0,0.0,4.2,NaN,0.0,0.0,27.5,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,24.6,0.0,0.0,0.0,0.0,0.0,14.1,4.3
2000-06-05,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,48.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-06-06,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-27,0.0,NaN,0.0,NaN,NaN,NaN,44.7,0.0,0.0,0.0,NaN,NaN,NaN,12.2,0.0,NaN,0.0,NaN,0.0,0.0,0.0,10.0,0.0,NaN,NaN,NaN,0.0,0.0,0.2,4.5,0.0,NaN,NaN,15.5,1.7,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,6.6,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,10.0,0.0,NaN,0.0,7.7,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,17.4,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,18.2,0.0,0.0,7.5,15.5,NaN,NaN
2015-12-28,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,8.4,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0

In [64]:
mylist = filter_gauges.values.tolist()
gauge_list = []
for l in mylist:
    gauge_list.append(l.split(','))
dfinho = pd.DataFrame(gauge_list)

In [65]:
lista2 = []
# percorrendo cada linha de instante de tempo de chuva
for i, list_gauges in enumerate(gauge_list):
    # pegando lista de chuvas nao nulas
    rainfall_values = gauge_data.iloc[i].dropna().tolist()
    # se tiver chuva em apenas uma estacao, nao precisa interpolar
    if len(rainfall_values)<2:
        continue
        
    final_idw = idw_all(list_gauges, rainfall_values, threshold, coords, alpha)
    lista2.append(final_idw)

In [66]:
lista2 = pd.DataFrame(lista2)
lista2 = lista2.rename(columns = {0: 'idw_values', 1: 'gauges', 2: 'unknow_id', 3: 'real_values'})

In [67]:
lista2['n_gauges'] = lista2['gauges'].apply(count_gauges)

In [78]:
def sMAE(sim, obs):
    lista_errors = []
    for s, o in zip(sim, obs):
        lista_errors.append(abs(o-s))
    return lista_errors

def calc_sMAE(x):
    return sMAE(x['idw_values'], x['real_values'])

def sMAPEabs(sim, obs):
    lista_errors = []
    for s, o in zip(sim, obs):
        lista_errors.append(abs(o-s))
    return lista_errors

def sMAPErel(sim, obs):
    lista_errors = []
    for s, o in zip(sim, obs):
        lista_errors.append(o+s)
    return lista_errors

def calc_sMAPEabs(x):
    return sMAPEabs(x['idw_values'], x['real_values'])

def calc_sMAPErel(x):
    return sMAPErel(x['idw_values'], x['real_values'])

lista2['erro_abs'] = lista2.apply(lambda x: calc_sMAPEabs(x), axis=1)
lista2['erro_rel'] = lista2.apply(lambda x: calc_sMAPErel(x), axis=1)

In [79]:
lista2

,idw_values,gauges,unknow_id,real_values,n_gauges,erro,erro_abs,erro_rel
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[2054005], [1456003], [1854006], [1656001], [1556005], [2055003, 2055004], [1557005], [2056007], [2055001, 2055002], [2157003], [2055001, 1955000], [2055003], [1654001], [1456001], [1555008], [1656002], [1557001], [1555001], [1557000], [2054009], [1854003], [1654004], [1457000], [1954004], [1556000], [1954006], [2055004]]","[2054009, 1456001, 1854003, 1656002, 1556000, 2055001, 1457000, 2157003, 2055003, 2056007, 2055004, 2055002, 1654004, 1456003, 1555001, 1656001, 1557000, 1555008, 1557001, 2054005, 1854006, 1654001, 1557005, 1954006, 1556005, 1954004, 1955000]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"[3.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8715582133482793, 0.0, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 4.1, 0.0, 15.5, 2.4, 0.0, 0.0, 0.0, 0.0]","[[2054005], [1456003], [1854006], [1656001], [1556005], [2055003, 2055004], [1557005], [2056007], [2055001, 2055002], [2157003], [2055001, 1955000], [2055003], [1654001], [1456001], [1555008], [1656002], [1557001], [1555001], [1557000], [2054009], [1854003], [1654004], [1457000], [1954004], [1556000], [1954006], [2055004]]","[2054009, 1456001, 1854003, 1656002, 1556000, 2055001, 1457000, 2157003, 2055003, 2056007, 2055004, 2055002, 1654004, 1456003, 1555001, 1656001, 1557000, 1555008, 1557001, 2054005, 1854006, 1654001, 1557005, 1954006, 1556005, 1954004, 1955000]","[4.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0]","[1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0.5999999999999996, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.7, 0.0, 0.0, 1.8715582133482793, 0.0, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 0.5999999999999996, 0.0, 15.5, 2.4, 0.0, 0.0, 0.0, 3.0]","[0.5999999999999996, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.7, 0.0, 0.0, 1.8715582133482793, 0.0, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 0.5999999999999996, 0.0, 15.5, 2.4, 0.0, 0.0, 0.0, 3.0]","[7.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.7, 0.0, 0.0, 1.8715582133482793, 0.0, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 7.6, 0.0, 15.5, 2.4, 0.0, 0.0, 0.0, 3.0]"
2,"[4.2, 0.0, 0.0, 0.0, 0.0, 3.869010773785017, 0.0, 20.2, 0.823417840904252, 18.9, 2.682566772465867, 0.0, 0.0, 0.0, 0.0, 7.500000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.1, 0.0, 0.0, 0.0, 0.0]","[[2054005], [1456003], [1854006], [1656001], [1556005], [2055003, 2055004], [1557005], [2056007], [2055001, 2055002], [2157003], [2055001, 1955000], [2055003], [1654001], [1456001], [1555008], [1656002], [1557001], [1555001], [1557000], [2054009], [1854003], [1654004], [1457000], [1954004], [1556000], [1954006], [2055004]]","[2054009, 1456001, 1854003, 1656002, 1556000, 2055001, 1457000, 2157003, 2055003, 2056007, 2055004, 2055002, 1654004, 1456003, 1555001, 1656001, 1557000, 1555008, 1557001, 2054005, 1854006, 1654001, 1557005, 1954006, 1556005, 1954004, 1955000]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.8, 0.0, 18.9, 5.8, 20.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.3]","[1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[4.2, 0.0, 0.0, 0.0, 0.0, 2.069010773785017, 0.0, 1.3000000000000007, 4.976582159095748, 1.3000000000000007, 

In [87]:
def convert2tuple(x):
    tuplinha = []
    for qtde, erro_abs, erro_rel, erro in zip(x['n_gauges'], x['erro_abs'], x['erro_rel'],  x['erro_rel']):
        tuplinha.append((qtde, erro_abs, erro_rel, erro))
    return tuplinha

lista2['dicio'] = lista2.apply(lambda x: convert2tuple(x), axis=1)

In [88]:
df = lista2[['dicio']].explode('dicio').reset_index().drop('index', axis=1)

In [95]:
df[['qtde','erro_abs','erro_rel','erro']] = pd.DataFrame(df['dicio'].tolist(), index=df.index)
df2 = df.drop('dicio', axis=1)
erro = df2.to_excel('erro_30km.xlsx', index = False)

In [99]:
df2

,qtde,erro_abs,erro_rel,erro
0,1,0.0,0.0,0.0
1,1,0.0,0.0,0.0
2,1,0.0,0.0,0.0
3,1,0.0,0.0,0.0
4,1,0.0,0.0,0.0
...,...,...,...,...
191194,1,18.6,18.6,18.6
191195,1,47.7,64.3,64.3
191196,1,35.8,35.8,35.8
191197,1,47.7,64.3,64.3


In [100]:
#sMAPE = pd.DataFrame(df2.groupby('qtde').mean())

sMAPE = pd.DataFrame(df2.groupby('qtde').sum().reset_index())
sMAPE['erro'] = sMAPE['erro_abs']/sMAPE['erro_rel']
sMAPE
sMAPE = sMAPE.to_excel('sMAPE.xlsx',index = False)

,qtde,erro_abs,erro_rel,erro
0,1,705330.100000,1.129925e+06,0.624228
1,2,254925.646072,4.295323e+05,0.593496
2,3,19516.838076,3.722157e+04,0.524342
